# AI 이용 주식 종목 추천

In [4]:
# !pip install openai
# !pip install python-dotenv

In [1]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

import yfinance as yf
import pandas as pd

In [2]:
# 주식 종목 리스트 (예: Tesla, 애플 등)
stocks = ['AAPL', 'GOOG', 'MSFT', 'TSLA', 'NVDA', 'AMZN', 'LLY']

# 주식 데이터 다운로드 (yfinance를 사용)
def download_stock_data(ticker, start_date="2024-01-01"):
    stock_data = yf.download(ticker, start=start_date)
    return stock_data

# 주식 데이터 수집
stock_data = {}
for stock in stocks:
    stock_data[stock] = download_stock_data(stock)

stock_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


{'AAPL':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2024-01-02  187.149994  188.440002  183.889999  185.639999  184.734970   
 2024-01-03  184.220001  185.880005  183.429993  184.250000  183.351746   
 2024-01-04  182.149994  183.089996  180.880005  181.910004  181.023163   
 2024-01-05  181.990005  182.759995  180.169998  181.179993  180.296707   
 2024-01-08  182.089996  185.600006  181.500000  185.559998  184.655365   
 ...                ...         ...         ...         ...         ...   
 2024-11-11  225.000000  225.699997  221.500000  224.229996  224.229996   
 2024-11-12  224.550003  225.589996  223.360001  224.229996  224.229996   
 2024-11-13  224.009995  226.649994  222.759995  225.119995  225.119995   
 2024-11-14  225.020004  228.869995  225.000000  228.220001  228.220001   
 2024-11-15  225.919998  226.919998  224.270004  225.000000  225.000000   
 
               

In [3]:
# 상승 가능성 판단을 위한 간단한 분석 (예: 이동평균 분석)
def calculate_moving_averages(stock_data):
    stock_data['SMA20'] = stock_data['Close'].rolling(20).mean()
    stock_data['SMA50'] = stock_data['Close'].rolling(50).mean()
    # 현재 SMA20 > SMA50인 경우 상승 가능성 있다고 판단
    return stock_data['SMA20'].iloc[-1] > stock_data['SMA50'].iloc[-1]

# 상승 가능성 높은 종목 리스트
potential_rising_stocks = []
for stock, data in stock_data.items():
    if calculate_moving_averages(data):
        potential_rising_stocks.append(stock)

potential_rising_stocks

['AAPL', 'GOOG', 'TSLA', 'NVDA', 'AMZN']

In [4]:
import yfinance as yf

# financials 딕셔너리 생성
financials = {}

# 상승 가능성이 있는 주식들의 티커 리스트(potential_rising_stocks)에서 각 주식의 재무 상태를 가져옵니다.
for ticker in potential_rising_stocks:
    # 각 티커에 해당하는 주식 데이터를 yfinance에서 불러옵니다.
    stock = yf.Ticker(ticker)
    
    # 해당 주식의 재무 정보를 저장할 딕셔너리 구조를 생성합니다.
    financials[ticker] = {}
    
    # 손익계산서(Income Statement) 데이터를 가져와서 financials 딕셔너리에 저장합니다.
    financials[ticker]['income statement'] = stock.financials
    
    # 대차대조표(Balance Sheet) 데이터를 가져와서 financials 딕셔너리에 저장합니다.
    financials[ticker]['balance sheet'] = stock.balance_sheet
    
    # 현금흐름표(Cash Flow Statement) 데이터를 가져와서 financials 딕셔너리에 저장합니다.
    financials[ticker]['cash flow'] = stock.cashflow

# 저장된 financials 딕셔너리의 키(티커 목록)를 출력합니다.
print(financials.keys())

# 전체 재무 데이터를 출력합니다.
financials

dict_keys(['AAPL', 'GOOG', 'TSLA', 'NVDA', 'AMZN'])


{'AAPL': {'income statement':                                                         2024-09-30  \
  Tax Effect Of Unusual Items                                    0.0   
  Tax Rate For Calcs                                        0.240912   
  Normalized EBITDA                                   134661000000.0   
  Net Income From Continuing Operation Net Minori...   93736000000.0   
  Reconciled Depreciation                              11445000000.0   
  Reconciled Cost Of Revenue                          210352000000.0   
  EBITDA                                              134661000000.0   
  EBIT                                                123216000000.0   
  Net Interest Income                                            NaN   
  Interest Expense                                               NaN   
  Interest Income                                                NaN   
  Normalized Income                                    93736000000.0   
  Net Income From Continuing And Dis

In [5]:
# OpenAI API로 상승 가능성 높은 종목에 대한 분석 요청
# 분석할 주식 리스트와 재무 상태를 기반으로 프롬프트를 작성합니다.
prompt = f"다음 주식들은 기술적 지표에 기반하여 상승 가능성이 있습니다: {', '.join(potential_rising_stocks)}. " + \
         "각 주식의 재무 상태는 다음과 같습니다:\n"

# 상승 가능성이 높은 종목의 재무 상태를 프롬프트에 추가합니다.
for ticker in potential_rising_stocks:
    prompt += f"\n{ticker}의 재무 상태:\n{financials[ticker]}\n"

# 프롬프트에 주식 추천 요청을 추가합니다.
prompt += "이 주식들 중에 어떤 주식이 가장 투자에 유망한지 두 종목을 추천해 주고 그 이유를 설명해 주세요. 한글로 답해주세요."

# OpenAI API로 질문을 보내고, 분석 결과를 받습니다.
response = client.chat.completions.create(
    model=Model,  # 사용할 모델을 지정합니다.
    messages=[
        {"role": "system", "content": "You are a helpful investment professional."},  # 시스템 역할 정의
        {"role": "user", "content": prompt},  # 사용자의 질문을 프롬프트로 전달
    ]
)

# OpenAI로부터 받은 응답을 출력합니다.
print(response.choices[0].message.content)

두 종목을 추천드리겠습니다: **AAPL (Apple Inc.)**와 **NVDA (NVIDIA Corporation)**입니다. 

### 1. AAPL (Apple Inc.)
- **재무 안정성**: AAPL은 높은 수익성과 안정적인 현금 흐름을 가지고 있습니다. 2024년 9월 기준으로 순이익이 약 937억 달러로 강력한 실적을 보이고 있으며, 과거 몇 년 동안 지속적으로 증가세를 유지하고 있습니다.
- **시장 점유율**: Apple은 스마트폰, 태블릿, 컴퓨터 등 여러 기술 분야에서 높은 시장 점유율을 가지고 있어, 앞으로도 안정적인 수익원을 기대할 수 있습니다.
- **신 제품 출시**: Apple은 연례 신제품 발표와 혁신적인 제품 라인업으로 잘 알려져 있으며, 이는 향후 성장을 뒷받침할 중요한 요소입니다.

### 2. NVDA (NVIDIA Corporation)
- **산업 리더십**: NVIDIA는 반도체 및 인공지능 분야에서의 리더십을 보이고 있으며, 특히 GPU 시장에서 독보적인 위치를 차지하고 있습니다. 클라우드 컴퓨팅, 게임, 자율주행차 등 다양한 분야에서 수요가 증가하고 있습니다.
- **재무 성과**: 2024년 초 기준으로 순이익이 약 297억 달러로, 빠른 성장세를 보이고 있으며, 주요 제품군인 GPU의 수요 증가가 결정적인 요인으로 작용하고 있습니다.
- **기술 혁신**: NVIDIA는 AI 및 머신 러닝 분야에서 핵심 기술 제공자로 자리 잡고 있어 앞으로의 기술 발전에 큰 영향을 미칠 것으로 보입니다.

### 결론
AAPL과 NVDA는 각각의 산업에서 강력한 재무 기반과 높은 성장 가능성을 가지고 있습니다. 이러한 요소들로 인해, 두 종목 모두 향후 투자에 유망한 선택이 될 수 있다고 판단됩니다.
